### Step 0 : Import Libraries and Dependencies**

In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering, Birch
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

### Step 1:  Read csv file and create a DataFrame

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head()

In [ ]:
df_market_data.info()

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Step 2: Prepare the Data 
**Scale the DataFrame and create a new DataFrame that contains the scaled data**

In [ ]:
df_market_data.columns

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
market_data_scaled = StandardScaler().fit_transform(df_market_data[['price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d',
       'price_change_percentage_60d', 'price_change_percentage_200d',
       'price_change_percentage_1y']])

In [ ]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(
    market_data_scaled, columns= ['price_change_percentage_24h', 'price_change_percentage_7d',
       'price_change_percentage_14d', 'price_change_percentage_30d',
       'price_change_percentage_60d', 'price_change_percentage_200d',
       'price_change_percentage_1y'])

# Copy the crypto names from the original DataFrame
df_market_data_scaled['coin_id'] = df_market_data.index

# Set the coin_id column as index
df_market_data_scaled = df_market_data_scaled.set_index('coin_id')

# Display the scaled DataFrame
df_market_data_scaled.head()

In [ ]:
df_market_data_scaled.describe()

After standardizing data the mean values are closer to zero and standard deviation for each feature is approximately 1.012423 which confirms that scaled data is centered around 0 with unit variance.

---

### Find the Best Value for k Using the Original Data.

In [ ]:
# Create a list with the number of k-values from 1 to 11
k= list(range(1,11))

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Create an empty list to store the inertia values
inertia=[]

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    k_model = KMeans(n_clusters=i,random_state=1)
    k_model.fit(df_market_data_scaled)
    inertia.append(k_model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {'k':k,'inertia':inertia}

# Create a DataFrame with the data to plot the Elbow curve
elbow_df = pd.DataFrame(elbow_data)
elbow_df.head()

In [ ]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
elbow_original_scaled = elbow_df.hvplot.line(x='k',
                     y='inertia',
                     title='Elbow Curve - Original Scaled Data',
                     xticks='k')
elbow_original_scaled

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** The resulting elbow graph shows two sharp points, out of which 4 seems to be the optimal K value with lower inertia of 79.022.

---

### Cluster Cryptocurrencies with K-means Using the Original Scaled DataFrame

In [ ]:
# Initialise the K-Means model using the best value for k
k_model = KMeans(n_clusters=4,random_state=1)

In [ ]:
# Fit the K-Means model using the scaled DataFrame
k_model.fit(df_market_data_scaled)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled DataFrame
k_model_clusters = k_model.predict(df_market_data_scaled)

# Print the resulting array of cluster values.
print(k_model_clusters)

In [ ]:
# Create a copy of the scaled DataFrame
k_model_prediction = df_market_data_scaled.copy()

In [ ]:
# Add a new column to the copy of the scaled DataFrame with the predicted clusters
k_model_prediction['k_clusters_4'] = k_model_clusters

# Display the copy of the scaled DataFrame
k_model_prediction.head()

In [ ]:
# Create a scatter plot using hvPlot by setting
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`.
# Colour the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.

sctter_original_scaled = k_model_prediction.hvplot.scatter(
x='price_change_percentage_24h',
y='price_change_percentage_7d',
by='k_clusters_4',
title="Cryptocurrency Clusters- Original Scaled Data :K=4 ",
hover_cols='coin_id'
)
sctter_original_scaled

This scatter plot presents a clearer distribution of cryptocurrencies based on their price change behaviors, showing four distinct clusters.

The Yellow cluster represents cryptocurrencies that are consistently increasing over both timeframes, short term (24 hrs) and long term (7 days) indicating steady growth.

The Blue cluster contains cryptocurrencies that are either stable or experiencing slight decline, especially over 24 hours. 

The Green cluster shows highly volatile behavior with a dramatic decline in 24 hours but recovery over 7 days, indicating sudden market shocks.

The Red cluster consists of a very small number of points, with modest +ve change over 24 hours and slightly +ve over 7 day period.

This scatter plot reflects a slight positive correlation between 24 hours and 7 days price change, specially for orange cluster.

---

### Optimise Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce the original scaled Dataframe
# down to three principal components.
market_data_pca = pca.fit_transform(df_market_data_scaled)

# View the scaled PCA data
market_data_pca[:5]

In [ ]:
# Retrieve the explained variance to determine how much information
# can be attributed to each principal component.
pca.explained_variance_ratio_

In [ ]:
sum(pca.explained_variance_ratio_)*100

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The first, second and third principal components explain 37.2% , 34.7 and 17.6% of the variance respectively. The relatively even distribution across the first two components indicates that both are almost equally important in explaining the variability in the data. 

This PCA model with 3 principal components has captured 89.50% of variability in the original features, which means these three components effectively capture the most significant patterns in the dataset, providing a well-balanced approach to dimensionality reduction and information retention.

In [ ]:
# Create a new DataFrame with the PCA data.
df_market_pca = pd.DataFrame(market_data_pca,columns=["PCA1",'PCA2','PCA3'])

# Copy the crypto names from the original scaled DataFrame
df_market_pca['coin_id'] = df_market_data.index

# Set the coin_id column as index
df_market_pca = df_market_pca.set_index('coin_id')

# Display the scaled PCA DataFrame
df_market_pca.head(10)

---

### Find the Best Value for k Using the Scaled PCA DataFrame

In [ ]:
# Create a list with the number of k-values from 1 to 11
k_pca=list(range(1,11))

In [ ]:
# Create an empty list to store the inertia values
inertia =[]

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k_pca:
    k_model = KMeans(n_clusters=i,random_state=0)
    k_model.fit(df_market_pca)
    inertia.append(k_model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {'k':k_pca, 'inertia':inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_pca = pd.DataFrame(elbow_data)
df_elbow_pca.head()

In [ ]:
# Plot a line chart with all the inertia values computed with
# the different values of k to visually identify the optimal value for k.
elbow_pca_scaled = df_elbow_pca.hvplot.line(x='k',
                    y='inertia',
                    title='Elbow Curve- PCA Data')
elbow_pca_scaled

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:** The sharp elbow appears at K=4 where the drop in inertia starts to slightly flatten out. So the optimal k value is 4 using PCA data.


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** Optimal K value ,determined from both the data sources ,original data and PCA scaled data,consistently reflects K=4. However, k=2 also looks like a candidate with sharp elbow.

### Cluster Cryptocurrencies with K-means Using the Scaled PCA DataFrame

In [ ]:
# Initialise the K-Means model using the best value for k
pca_model = KMeans(n_clusters=4,random_state=1)

In [ ]:
# Fit the K-Means model using the scaled PCA DataFrame
pca_model.fit(df_market_pca)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled PCA DataFrame
pca_clusters = pca_model.predict(df_market_pca)

# Print the resulting array of cluster values.
print(pca_clusters)

In [ ]:
# Create a copy of the scaled PCA DataFrame
market_pca_prediction = df_market_pca.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
market_pca_prediction['pca_clusters_4'] = pca_clusters

# Display the copy of the scaled PCA DataFrame
market_pca_prediction.head()

In [ ]:
# Create a scatter plot using hvPlot by setting
# `x="PC1"` and `y="PC2"`.
# Colour the graph points with the labels found using K-Means and
# add the crypto name in the `hover_cols` parameter to identify
# the cryptocurrency represented by each data point.

scatter_pca_scaled = market_pca_prediction.hvplot.scatter(x='PCA1',
                            y='PCA2',
                            by='pca_clusters_4',
                            hover_cols='coin_id',
                                    title="CryptoCurrency Segment- Scaled PCA data: PCA=3,K=4")
scatter_pca_scaled

### Visualise and Compare the Results

In this section, you will visually analyse the cluster analysis results by contrasting the outcome with and without using the optimisation techniques.

The PCA-based clustering effectively grouped cryptocurrencies into four distinct clusters, each exhibiting clear behavioral patterns such as stability, growth, and volatility. While k-means clustering without optimization also identified meaningful clusters, it was less effective at capturing distinct differences, particularly in high-dimensional data. Notably, the green and red clusters contained only a single data point, prompting further analysis using alternative clustering models and performance comparisons to achieve a more accurate and optimal k value

### Getting the Cryptocurrency from original data for each cluster based on PCA

In [ ]:
#Inspect the DataFrame with assigned clusters
PCA_cryptocurrencies_df = market_pca_prediction.reset_index()

#Retrieve cryptocurrencies coin_id by cluster
for cluster_label in PCA_cryptocurrencies_df['pca_clusters_4'].unique():
    coin_ids_in_cluster = PCA_cryptocurrencies_df[PCA_cryptocurrencies_df['pca_clusters_4'] == cluster_label]['coin_id']
    print(f"Cluster {cluster_label} Cryptocurrencies: {coin_ids_in_cluster.values}")
    print("\n")

PCA_cryptocurrencies_df.head()

### Clusterring the data using AgglomerativeClustering and Birch

In [ ]:
birch_model = Birch(n_clusters=4)
birch_predictions = birch_model.fit_predict(df_market_data_scaled)
birch_predictions[:5]

In [ ]:
agglo_model =  AgglomerativeClustering(n_clusters=4)
agglo_predictions = agglo_model.fit_predict(df_market_data_scaled)
agglo_predictions[:5]

### Compare the cluster results from using Kmeans, AgglomerativeClustering, Birch and PCA

In [ ]:
#Creting new DataFrame to store clusters from individual Clustering models
market_prediction_all = df_market_data_scaled.copy()

market_prediction_all['KMeans_Segments'] = k_model_clusters
market_prediction_all['Agglomerative_Segments'] = agglo_predictions
market_prediction_all['Birch_Segments'] = birch_predictions
market_prediction_all['PCA_Segments']= pca_clusters

market_prediction_all[['KMeans_Segments','Agglomerative_Segments','Birch_Segments','PCA_Segments']].head()

In [ ]:
#Correlation Analysis of Segments:
market_prediction_all[['KMeans_Segments','Agglomerative_Segments','Birch_Segments','PCA_Segments']].corr()

**Correlation Analysis of Segments**

KMeans and PCA show perfect correlation. This suggests that the clusters derived from KMeans are heavily aligned with the principal components, which likely means that the PCA-reduced data retains most of the clustering structure.

Similarly, Agglomerative and Birch are perfectly correlated, showing that both hierarchical methods identify identical clusters.

Given the perfect correlation between KMeans and PCA, as well as Agglomerative and Birch, it would be redundant to use both KMeans and PCA or Agglomerative and Birch in the final analysis.

### Comparing the models based on the Calinski-Harabasz Score 

In [ ]:
from sklearn import metrics

# Create a list to store the scores
score_kmeans = []
score_agglomerative = []
score_birch = []

# Create a list of k values to test (number of clusters)
k = list(range(2, 11))

for i in k:
    # KMeans clustering
    kmeans_model = KMeans(n_clusters=i, random_state=0)
    kmeans_labels = kmeans_model.fit_predict(df_market_data_scaled)
    kmeans_score = metrics.calinski_harabasz_score(df_market_data_scaled, kmeans_labels)
    score_kmeans.append(kmeans_score)
    
    # Agglomerative Clustering
    agglo_model = AgglomerativeClustering(n_clusters=i)
    agglo_labels = agglo_model.fit_predict(df_market_data_scaled)
    agglo_score = metrics.calinski_harabasz_score(df_market_data_scaled, agglo_labels)
    score_agglomerative.append(agglo_score)
    
    # Birch Clustering
    birch_model = Birch(n_clusters=i)
    birch_labels = birch_model.fit_predict(df_market_data_scaled)
    birch_score = metrics.calinski_harabasz_score(df_market_data_scaled, birch_labels)
    score_birch.append(birch_score)

In [ ]:
# Combine scores into a DataFrame for easy plotting with hvPlot
df_scores = pd.DataFrame({
    'K_Calinski': k,
    'KMeans_Calinski': score_kmeans,
    'Agglomerative_Calinski': score_agglomerative,
    'Birch_Calinski': score_birch
})
df_scores.set_index('K_Calinski')

**Perfomance Analysis based on Calinski-Harabasz Score**

Agglomerative Clustering consistently has the highest scores across most K values, especially for K=5 and higher. This suggests that Agglomerative Clustering forms the most distinct and well-separated clusters overall.

**KMeans performs well when K=4, where it achieves the highest score of 32.46. Which is the highest among all the scors. However, for K=5 and above, its performance is generally lower than Agglomerative Clustering.**

Birch shows competitive performance for higher k values , K=6, k=7, but its performance declines slightly for lower K values and as K increases, making it less effective for larger numbers of clusters.

### Comparing the models based on the Silhouette scores

In [ ]:
# Create a list to store the Silhouette scores
silhouette_kmeans = []
silhouette_agglomerative = []
silhouette_birch = []

# Create a list of k values to test (number of clusters)
k_values = list(range(2, 11))

for k in k_values:
    # KMeans clustering
    kmeans_model = KMeans(n_clusters=k, random_state=0)
    kmeans_labels = kmeans_model.fit_predict(df_market_data_scaled)
    kmeans_silhouette = metrics.silhouette_score(df_market_data_scaled, kmeans_labels)
    silhouette_kmeans.append(kmeans_silhouette)
    
    # Agglomerative Clustering
    agglo_model = AgglomerativeClustering(n_clusters=k)
    agglo_labels = agglo_model.fit_predict(df_market_data_scaled)
    agglo_silhouette = metrics.silhouette_score(df_market_data_scaled, agglo_labels)
    silhouette_agglomerative.append(agglo_silhouette)
    
    # Birch Clustering
    birch_model = Birch(n_clusters=k)
    birch_labels = birch_model.fit_predict(df_market_data_scaled)
    birch_silhouette = metrics.silhouette_score(df_market_data_scaled, birch_labels)
    silhouette_birch.append(birch_silhouette)

# Combine Silhouette scores into a DataFrame for easy plotting with hvPlot
df_silhouette_scores = pd.DataFrame({
    'K_sil': k_values,
    'KMeans_sil': silhouette_kmeans,
    'Agglomerative_sil': silhouette_agglomerative,
    'Birch_sil': silhouette_birch
})

df_silhouette_scores

In [ ]:
#Combinigh the DataFrames to compare  Calinski-Harabasz scores and Silhouette scores
df_scores = pd.concat([df_scores,df_silhouette_scores],axis=1)
df_scores[['Agglomerative_sil', 'KMeans_sil','Birch_sil']] = df_scores[['Agglomerative_sil', 'KMeans_sil','Birch_sil']].round(2)*100
df_scores[['Agglomerative_sil', 'KMeans_sil','Birch_sil']] = df_scores[['Agglomerative_sil', 'KMeans_sil','Birch_sil']].astype(int)
df_scores

In [ ]:
# Convert the DataFrame into long format for hvPlot compatibility
df_silhouette_melted = df_silhouette_scores.melt(id_vars=['K_sil'], var_name='Model', value_name='Silhouette_Score')

# Plot using hvPlot
df_silhouette_melted.hvplot.line(x='K_sil', y='Silhouette_Score', by='Model', width=800, height=400, 
                      title='Model Comparison Based on Silhouette Score', xlabel='Number of Clusters (k)', 
                      ylabel='Silhouette Score')

**Observations for Silhouette Scores:**

KMeans performs best for K=2, achieving the highest silhouette score of 72. However, its performance significantly drops as K increases, especially for K=3, where it has a score of 34.

Agglomerative Clustering shows consistent silhouette scores across K=2 to K=4, but its performance dips for K greater than 5.

Birch has similar performance to Agglomerative for small K values (up to K=4), but it performs poorly as the number of clusters increases (K=6 to K=10).

**Conclusion Based on Silhouette Scores:
KMeans outperforms the other models for K=2, but for larger K values, it does not maintain good cluster separation.**

**Observations for Calinski-Harabasz Scores:**
KMeans performs well when K=4, where it achieves the highest score of 32.46. However, for K=5 and above, its performance is generally lower than Agglomerative Clustering.

**Observations for Silhouette Scores:**
KMeans performs best for K=2, achieving the highest silhouette score of 72. However, its performance significantly drops as K increases, especially for K=3, where it has a score of 34.

Based on both the scores Optimal value for K=2

#### Plotting clusters Cryptocurrencies with new K-means Using the PCA Data

In [ ]:
# K-Means Model using K=2
k_model = KMeans(n_clusters=2,random_state=1)
k_model.fit(df_market_data_scaled)
k_model_clusters = k_model.predict(df_market_data_scaled)
k_model_prediction = df_market_data_scaled.copy()
k_model_prediction['k_clusters_4'] = k_model_clusters
#k_model_prediction.head()

sctter_original_scaled_c2 = k_model_prediction.hvplot.scatter(
x='price_change_percentage_24h',
y='price_change_percentage_7d',
by='k_clusters_4',
title="Cryptocurrency Clusters- Original Scaled Data :K=4 ",
hover_cols='coin_id'
)
sctter_original_scaled_c2

In [ ]:
#Reducing the featurs based on PCA
pca_model = KMeans(n_clusters=2,random_state=1)
pca_model.fit(df_market_pca)
pca_clusters = pca_model.predict(df_market_pca)
print(pca_clusters)

# Create a copy of the scaled PCA DataFrame
market_pca_clusters = df_market_pca.copy()

# Add a new column to the copy of the PCA DataFrame with the predicted clusters
market_pca_prediction['pca_clusters_2'] = pca_clusters

# Display the copy of the scaled PCA DataFrame
market_pca_prediction.head()

scatter_pca_scaled_c2 = market_pca_prediction.hvplot.scatter(x='PCA1',
                            y='PCA2',
                            by='pca_clusters_2',
                            hover_cols='coin_id',
                                    title="CryptoCurrency Segment- Scaled PCA data: PCA=3,K=2")
scatter_pca_scaled_c2

#### Answer the following question: 

  * **Question:** After visually analysing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** After evaluating the Silhouette and Calinski-Harabasz scores, the results indicate that KMeans performs optimally with K=2, achieving a highest Silhouette score of 72. Applying K=2 and visualizing the cluster analysis revealed that reducing the number of features had a significant impact on the clustering outcome. Initially, the elbow curve suggested K=4 based on the original data. However, with the highest Silhouette score at K=2 and the application of PCA, the resulting plot demonstrated clearer and more precise clustering.

In [ ]:
# Composite plot to contrast the Elbow curves
composite_plot_elbow = elbow_original_scaled + elbow_pca_scaled
composite_plot_elbow.cols(2) # Arrange in two columns (side by side)
composite_plot_elbow.opts(title='Elbow Curve Comparison')
composite_plot_elbow

In [ ]:
# Composite plot to contrast the clusters K=4
composite_plot_scatter=sctter_original_scaled + scatter_pca_scaled
composite_plot_scatter.cols(2) # Arrange in two columns (side by side)
composite_plot_scatter.opts(title='Cryptocurrency Clusters Comparision  K=4')
composite_plot_scatter

In [ ]:
# Composite plot to contrast the clusters K=2
composite_plot_scatter_k2=sctter_original_scaled_c2 + scatter_pca_scaled_c2
composite_plot_scatter_k2.cols(2) # Arrange in two columns (side by side)
composite_plot_scatter_k2.opts(title='Cryptocurrency Clusters Comparision  K=2')
composite_plot_scatter_k2

#### Checking the Relationship between 4 Clusters and indivisual features

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

kmean_cluster_group = k_model_prediction.groupby('k_clusters_4').mean()

# Step 3: Create a heatmap using Seaborn
plt.figure(figsize=(10, 6))
sns.heatmap(kmean_cluster_group, annot=True, cmap="coolwarm", linewidths=0.5)

# Display the plot
plt.title('Heatmap of KMean Clusters vs. Features')
plt.xlabel('Features')
plt.ylabel('Cluster')
plt.show()

The heatmap visualizes the relationship between KMeans clusters and various features related to cryptocurrency price changes over different time periods (e.g., 24h, 7d, 30d). Here are some key insights:

Cluster 0 appears to have relatively stable values across all time periods, as indicated by the cool colors (blues), with little variation in price changes. This suggests that the assets in Cluster 0 show more stability over time.

Cluster 1, on the other hand, shows significant variation, particularly in the mid-term periods (14d, 30d, 60d, 200d), as indicated by the warmer colors (reds and oranges). This implies that the assets in Cluster 1 are experiencing greater volatility, particularly in these time frames.

The most significant difference between the two clusters is observed in the 30-day price change and 60-day price change, where Cluster 1 exhibits high levels of volatility.

In summary, Cluster 0 represents more stable assets, while Cluster 1 captures assets with higher volatility, particularly over mid-term periods. Further analysis could explore the specific behaviors driving this volatility.

In [ ]:
#Inspect the DataFrame with assigned clusters
market_pca_clusters_k2 = market_pca_prediction.reset_index()

#Inspect the DataFrame with assigned clusters
PCA_cryptocurrencies_df = market_pca_clusters_k2.reset_index()

#Retrieve cryptocurrencies coin_id by cluster
for cluster_label in market_pca_clusters_k2['pca_clusters_2'].unique():
    coin_ids_in_cluster = market_pca_clusters_k2[market_pca_clusters_k2['pca_clusters_2'] == cluster_label]['coin_id']
    print(f"Cluster {cluster_label} Cryptocurrencies: {coin_ids_in_cluster.values}")
    print("\n")

market_pca_clusters_k2.head()


### Interpretation of the PCA Component Loadings Heatmap

In [ ]:
# Step 2: Fit PCA
pca = PCA(n_components=3)  # Choose the number of components you want to retain
pca.fit(market_data_scaled)

# Step 3: Create a DataFrame to show which features contribute to each PCA component
# Access PCA components (weights of each feature for each principal component)
pca_components = pd.DataFrame(
    pca.components_, 
    columns=df_market_data.columns,  # Original feature names as column headers
    index=[f'PCA{i+1}' for i in range(pca.n_components_)]  # Name each component (PCA1, PCA2, ...)
)

# Display the DataFrame
print("PCA Component Loadings:")
print(pca_components)

# If you want to visualize it:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
sns.heatmap(pca_components, annot=True, cmap='coolwarm')
plt.title('PCA Component Loadings')
plt.xlabel('Features')
plt.ylabel('Principal Components')
plt.show()


Feature Relationships: The heatmap provides insights into how original features relate to the underlying principal components derived through PCA.

PCA Component Utility: PCA1 and PCA2 capture essential market dynamics. PCA1 is crucial for short-term trends, while PCA2 is more reflective of mid-term performance.

Strategic Decisions: Investors can use this analysis to focus on short-term indicators (like 24h and 7d changes) when making quick trading decisions, while mid-term indicators (30d and 60d changes) might be more relevant for strategic positioning.

#### Final Analysis

The Mid-Term Cluster - yellow cluster - Cluster 2 (k=4) highlights promising investment opportunities in cryptocurrencies that demonstrate consistent growth across both short-term (24 hours) and long-term (7 days) timeframes. This cluster indicates steady upward momentum, making these assets particularly attractive for investors seeking reliable performance.

This includes notable cryptocurrencies such as **Bitcoin, Ethereum, Bitcoin Cash, Binance Coin, Chainlink, Cardano, Litecoin, Monero, Tezos, Cosmos, Wrapped Bitcoin, Zcash, and Maker.**  These assets show potential for sustained growth over a mid-term horizon, further supporting strategic investment decisions.





